<a href="https://colab.research.google.com/github/OSegun/Zummit-Africa-ML-AL-Projects/blob/main/Contract_Review_PR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
#!pip install torch
#!pip install transformers

In [2]:
!git clone https://github.com/TheAtticusProject/cuad.git

Cloning into 'cuad'...
remote: Enumerating objects: 30, done.
remote: Total 30 (delta 0), reused 0 (delta 0), pack-reused 30
Unpacking objects: 100% (30/30), 17.78 MiB | 12.65 MiB/s, done.


In [3]:
!wget https://zenodo.org/record/4599830/files/roberta-base.zip?download=1

--2023-05-15 11:11:19--  https://zenodo.org/record/4599830/files/roberta-base.zip?download=1
Resolving zenodo.org (zenodo.org)... 188.185.124.72
Connecting to zenodo.org (zenodo.org)|188.185.124.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 447429160 (427M) [application/octet-stream]
Saving to: ‘roberta-base.zip?download=1’

roberta-base.zip?do 100%[===================>] 426.70M  1.95MB/s    in 3m 28s  

2023-05-15 11:15:03 (2.05 MB/s) - ‘roberta-base.zip?download=1’ saved [447429160/447429160]



In [4]:
!unzip /content/cuad/data.zip

Archive:  /content/cuad/data.zip
  inflating: CUADv1.json             
  inflating: test.json               
  inflating: train_separate_questions.json  


In [5]:
!mkdir cuad-models

In [6]:
!unzip /content/roberta-base.zip?download=1 -d cuad-models/

Archive:  /content/roberta-base.zip?download=1
   creating: cuad-models/roberta-base/
  inflating: cuad-models/roberta-base/special_tokens_map.json  
  inflating: cuad-models/roberta-base/tokenizer_config.json  
  inflating: cuad-models/roberta-base/pytorch_model.bin  
  inflating: cuad-models/roberta-base/training_args.bin  
  inflating: cuad-models/roberta-base/vocab.json  
  inflating: cuad-models/roberta-base/predictions_.json  
  inflating: cuad-models/roberta-base/merges.txt  
  inflating: cuad-models/roberta-base/nbest_predictions_.json  
  inflating: cuad-models/roberta-base/null_odds_.json  
  inflating: cuad-models/roberta-base/config.json  


In [ ]:
#https://zenodo.org/record/4599830/files/roberta-base.zip?download=1 --output cuad-models/roberta-base.zip

In [7]:
import torch
import time
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import AutoConfig, AutoModelForQuestionAnswering, AutoTokenizer, squad_convert_examples_to_features
from transformers.data.processors.squad import SquadResult, SquadV2Processor, SquadExample
from transformers.data.metrics.squad_metrics import compute_predictions_logits

In [8]:
def to_list(tensor):
    return tensor.detach().cpu().tolist()

In [9]:
config = AutoConfig.from_pretrained('/content/cuad-models/roberta-base')
tokenizer = AutoTokenizer.from_pretrained('/content/cuad-models/roberta-base', do_lower_case=True, use_fast=False)
model = AutoModelForQuestionAnswering.from_pretrained('/content/cuad-models/roberta-base', config=config)

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

RobertaForQuestionAnswering(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (Lay

In [11]:
import json

with open('/content/CUADv1.json') as json_file:
    data = json.load(json_file)

questions = []
for i, q in enumerate(data['data'][0]['paragraphs'][0]['qas']):
    question = data['data'][0]['paragraphs'][0]['qas'][i]['question']
    questions.append(question)
contract = data['data'][0]['paragraphs'][0]['context']

In [ ]:
questions

In [ ]:
contract

In [12]:
with open('contract.txt', 'w') as f:
    f.write(' '.join(contract.split()))

In [13]:
processor = SquadV2Processor()
question_result = []

for i, question in enumerate(questions):
        result = SquadExample(
            qas_id=str(i),
            question_text= question,
            context_text = contract,
            answer_text=None,
            start_position_character=None,
            title="Predict",
            answers=None,
        )

        question_result.append(result)

In [14]:
all_result = []

features, dataset = squad_convert_examples_to_features(
        examples=question_result,
        tokenizer=tokenizer,
        max_seq_length= 512,
        doc_stride= 256,
        max_query_length= 64,
        is_training=False,
        return_dataset="pt",
        threads=1,
    )

eval_sampler = SequentialSampler(dataset)
eval_dataloader = DataLoader(dataset, sampler=eval_sampler, batch_size=10)

add example index and unique id: 100%|██████████| 41/41 [00:00<00:00, 28145.08it/s]


In [16]:
for batch in eval_dataloader:
        model.eval()
        batch = tuple(t.to(device) for t in batch)

        with torch.no_grad():
            inputs = {
                "input_ids": batch[0],
                "attention_mask": batch[1],
                "token_type_ids": batch[2],
            }

            example_indices = batch[3]

            outputs = model(**inputs)

            for i, example_index in enumerate(example_indices):
                eval_feature = features[example_index.item()]
                unique_id = int(eval_feature.unique_id)

                output = [to_list(output[i]) for output in outputs.to_tuple()]

                start_logits, end_logits = output
                result = SquadResult(unique_id, start_logits, end_logits)
                all_result.append(result)

In [17]:
predictions = compute_predictions_logits(
        all_examples=question_result,
        all_features=features,
        all_results=all_result,
        n_best_size= 1,
        max_answer_length= 512,
        do_lower_case= False,
        output_prediction_file=None,
        output_nbest_file=None,
        output_null_log_odds_file=None,
        verbose_logging=False,
        version_2_with_negative=True,
        null_score_diff_threshold= 0.0,
        tokenizer=tokenizer)

In [18]:
predictions

OrderedDict([('0', 'DISTRIBUTOR AGREEMENT'),
             ('1', 'Electric City of Illinois L.L.C.'),
             ('2', '7th day of September, 1999.'),
             ('3',
              'The term of this Agreement shall be ten (10) years (the "Term") which shall commence on the date upon which the Company delivers to Distributor the last Sample, as defined hereinafter.'),
             ('4',
              'The term of this Agreement shall be ten (10) years (the "Term") which shall commence on the date upon which the Company delivers to Distributor the last Sample, as defined hereinafter.'),
             ('5',
              'If Distributor complies with all of the terms of this Agreement, the Agreement shall be renewable on an annual basis for one (1) year terms for up to another ten (10) years on the same terms and conditions as set forth herein.'),
             ('6',
              'If Distributor complies with all of the terms of this Agreement, the Agreement shall be renewable on an an

In [19]:
with open('contract_review.txt', 'w') as f:
    for i, p in enumerate(predictions):
        f.write(f"Question {i+1}: {questions[int(p)]}\nAnswer: {predictions[p]}\n\n")

In [ ]:
with open('/content/test.json') as json_file:
    data = json.load(json_file)

test = []
for i, q in enumerate(data['data'][0]['paragraphs'][0]['qas']):
    question = data['data'][0]['paragraphs'][0]['qas'][i]['question']
    test.append(question)

In [ ]:
with open('/content/train_separate_questions.json') as json_file:
    data = json.load(json_file)

s_question = []
for i, q in enumerate(data['data'][0]['paragraphs'][0]['qas']):
    question = data['data'][0]['paragraphs'][0]['qas'][i]['question']
    s_question.append(question)

s_question